# Формулировка проблемы

Low-level wrapper для Python должен являться **биндингом** имеющегося API в Питон, учитывающим паттерны программирования в Питоне. При этом он не должен являться самостоятельной библиотекой, требующей поддержки и диктующей свои правила, как это происходит с существующий Low-level Python API.

В идеале:
  - человек по [документации C-API](http://docs.bigartm.org/en/latest/ref/c_interface.html) должен без дополнительных знаний брать и пользоваться оберткой в питоне
  - при изменении C-API необходимо мимнимум правок в Python-обертку
  - у пользователя не должно быть попоболи от жутких конструкций, не свойственных Python
  
Предлагаемое решение:
Обертка C-API для питона должна генерироваться автоматчиески по достаточно простому описанию. Обертка выполняется в виде одного небольшого модуля (не более 1к строк, лучше — меньше), содержащего класс `LibArtm`. 

На основе Low-level wrapper будет строиться удобная python-библиотека для тематического моделирования (реализация `ArtmModel`).

# Использование C-API напрямую

In [1]:
import ctypes
from artm import messages_pb2

# Сообщения об ошибках

In [35]:
ARTM_SUCCESS = 0
ARTM_STILL_WORKING = -1

class ArtmException(BaseException): pass

class InternalError(ArtmException): pass
class ArgumentOutOfRangeException(ArtmException): pass
class InvalidMasterIdException(ArtmException): pass
class CorruptedMessageException(ArtmException): pass
class InvalidOperationException(ArtmException): pass
class DiskReadException(ArtmException): pass
class DiskWriteException(ArtmException): pass

ARTM_EXCEPTION_BY_CODE = {
    -2: InternalError,
    -3: ArgumentOutOfRangeException,
    -4: InvalidMasterIdException,
    -5: CorruptedMessageException,
    -6: InvalidOperationException,
    -7: DiskReadException,
    -8: DiskWriteException,
}

# Обертка для удобного вызова низкоуровнего API

In [46]:
import ctypes
from google import protobuf
from artm import messages_pb2

class LibArtm(object):
    
    def __init__(self, lib_name):
        self.cdll = ctypes.CDLL(lib_name)
        
    def __getattr__(self, name):
        func = getattr(self.cdll, name)
        if func is None:
            raise AttributeError('%s is not a function of libartm' % name)
        return self._wrap_call(func)
    
    def _check_error(self, error_code):
        if error_code < -1:
            lib.cdll.ArtmGetLastErrorMessage.restype = ctypes.c_char_p
            error_message = lib.cdll.ArtmGetLastErrorMessage()
            
            # remove exception name from error message
            error_message = error_message.split(':', 1)[-1].strip()
            
            exception_class = ARTM_EXCEPTION_BY_CODE.get(error_code)
            if exception_class is not None:
                raise exception_class(error_message)
            else:
                raise RuntimeError(error_message)
    
    def _wrap_call(self, func):
        def artm_api_call(*args):
            cargs = []
            for arg in args:
                if isinstance(arg, basestring):
                    arg_cstr_p = ctypes.create_string_buffer(arg)
                    cargs.append(arg_cstr_p)
                    
                elif isinstance(arg, protobuf.message.Message):
                    message_str = arg.SerializeToString()
                    message_cstr_p = ctypes.create_string_buffer(message_str)
                    cargs += [len(message_str), message_cstr_p]
                    
                else:
                    cargs.append(arg)
            
            result = func(*cargs)
            self._check_error(result)
            return result
        
        return artm_api_call
            
    def _copy_request_result(self, length):
        message_blob = ctypes.create_string_buffer(length)
        error_code = self.lib_.ArtmCopyRequestResult(length, message_blob)
        self._check_error(error_code)
        return message_blob
        

In [47]:
lib = LibArtm('build/src/artm/libartm.dylib')

In [49]:
config = messages_pb2.MasterComponentConfig()
config.processors_count = -1

master_id = lib.ArtmCreateMasterComponent(config)

In [50]:
lib.ArtmCreateModel(master_id, messages_pb2.ModelConfig())

0

# Спецификация API и автоматическая генерация обертки

Модификация обертки, приведенной выше, решает следующие задачи:

 - Автоматическа проверка аргументов
 - Осмысленное возвращаемое значение: 
   - `ArtmCreateMasterComponent` возвращает `int`
   - функции вроде `ArtmRequestThetaMatrix` возвращают сообщение
   - функции без возвращаемого знвчения возвращают `None` (вместо `ARTM_SUCCESS`)
 - Автодокументирование: при вызове справки, отображается нормальная спецификация функции
 - Быстрое создание сообщений: вместо сообщения можно передать `dict`, который умным образом преобразуется в соответствующее сообщение (гораздо удобнее в питоне)

##### Вводим элементы языка, которыми будет описано API

In [52]:
class CallSpec(object):
    def __init__(self, name, arguments, result=None):
        self.name_ = name
        self.arguments_ = arguments
        self.result_ = result
        
class Arguments(object):
    def __init__(self, **kwargs):
        self.fields = [
            (name, type_)
            for name, type_ in kwargs.iteritems()
        ]

##### Описываем API при помощи специального dsl

Список `ARTM_API` нужно будет редактировать при изменении low-level API. Заметьте: необходимо будет минимальное число правок, в отличие от существующего `artm.library`.

Некоторые функции из API объявляются _специальными_, для них по понятной причине не делается обертка:
  - `ArtmGetLastErrorMessage`
  - `ArtmCopyRequestResult`

In [53]:
ARTM_API = [
    CallSpec(
        'ArtmCreateMasterComponent', 
        Arguments(config=messages_pb2.MasterComponentConfig), 
        int,
    ),
    CallSpec(
        'ArtmReconfigureMasterComponent',
        Arguments(master_id=int, config=messages_pb2.MasterComponentConfig),
    ),
    CallSpec(
        'ArtmDisposeMasterComponent',
        Arguments(master_id=int),
    ),
    CallSpec(
        'ArtmCreateModel',
        Arguments(master_id=int, config=messages_pb2.ModelConfig),
    ),
    CallSpec(
        'ArtmReconfigureModel',
        Arguments(master_id=int, config=messages_pb2.ModelConfig),
    ),
    CallSpec(
        'ArtmDisposeModel',
        Arguments(master_id=int, name=str),
    ),
    CallSpec(
        'ArtmCreateRegularizer',
        Arguments(master_id=int, config=messages_pb2.RegularizerConfig),
    ),
    CallSpec(
        'ArtmReconfigureRegularizer',
        Arguments(master_id=int, config=messages_pb2.RegularizerConfig),
    ),
    CallSpec(
        'ArtmDisposeRegularizer',
        Arguments(master_id=int, name=str),
    ),
    CallSpec(
        'ArtmCreateDictionary',
        Arguments(master_id=int, config=messages_pb2.DictionaryConfig),
    ),
    CallSpec(
        'ArtmReconfigureDictionary',
        Arguments(master_id=int, config=messages_pb2.DictionaryConfig),
    ),
    CallSpec(
        'ArtmDisposeDictionary',
        Arguments(master_id=int, name=str),
    ),
    CallSpec(
        'ArtmAddBatch',
        Arguments(master_id=int, args=messages_pb2.AddBatchArgs),
    ),
    CallSpec(
        'ArtmInvokeIteration',
        Arguments(master_id=int, args=messages_pb2.InvokeIterationArgs),
    ),
    CallSpec(
        'ArtmSynchronizeModel',
        Arguments(master_id=int, args=messages_pb2.SynchronizeModelArgs),
    ),
    CallSpec(
        'ArtmInitializeModel',
        Arguments(master_id=int, args=messages_pb2.InitializeModelArgs),
    ),
    CallSpec(
        'ArtmExportModel',
        Arguments(master_id=int, args=messages_pb2.ExportModelArgs),
    ),
    CallSpec(
        'ArtmImportModel',
        Arguments(master_id=int, args=messages_pb2.ImportModelArgs),
    ),
    CallSpec(
        'ArtmWaitIdle',
        Arguments(master_id=int, args=messages_pb2.WaitIdleArgs),
    ),
    CallSpec(
        'ArtmOverwriteTopicModel',
        Arguments(master_id=int, model=messages_pb2.TopicModel),
    ),
    CallSpec(
        'ArtmRequestThetaMatrix',
        Arguments(master_id=int, args=messages_pb2.GetThetaMatrixArgs),
        messages_pb2.ThetaMatrix,
    ),
    CallSpec(
        'ArtmRequestTopicModel',
        Arguments(master_id=int, args=messages_pb2.GetTopicModelArgs),
        messages_pb2.TopicModel,
    ),
    CallSpec(
        'ArtmRequestRegularizerState',
        Arguments(master_id=int, name=str),
        messages_pb2.RegularizerInternalState,
    ),
    CallSpec(
        'ArtmRequestScore',
        Arguments(master_id=int, args=messages_pb2.GetScoreValueArgs),
        messages_pb2.ScoreData,
    ),
    CallSpec(
        'ArtmRequestParseCollection',
        Arguments(args=messages_pb2.CollectionParserConfig),
        messages_pb2.DictionaryConfig,
    ),
    CallSpec(
        'ArtmRequestLoadDictionary',
        Arguments(filename=str),
        messages_pb2.DictionaryConfig,
    ),
    CallSpec(
        'ArtmRequestLoadBatch',
        Arguments(filename=str),
        messages_pb2.Batch,
    ),
    CallSpec(
        'ArtmSaveBatch',
        Arguments(filename=str, batch=messages_pb2.Batch),
        ErrorCode,
    ),
    CallSpec(
        'ArtmSaveBatch',
        Arguments(filename=str, batch=messages_pb2.Batch),
        ErrorCode,
    ),
]

##### Объект класса `LibArtm` при помощи `__getattr__` автоматически создает обертки над вызовами API

In [ ]:
class LibArtm(object):
    pass # TODO